# AutoKaggle: Machine Learning Tools Library

## Mục Tiêu

Notebook này tập trung vào phân tích chuyên sâu về thư viện công cụ học máy (Machine Learning Tools Library) được giới thiệu trong bài báo "AutoKaggle: A Multi-Agent Framework for Autonomous Data Science Competitions". Chúng ta sẽ khám phá cách thiết kế, tổ chức và triển khai một bộ công cụ toàn diện hỗ trợ toàn bộ quy trình khoa học dữ liệu.

Sau khi hoàn thành notebook này, bạn sẽ:
1. Hiểu được cách tổ chức thư viện công cụ ML theo các danh mục chức năng
2. Nắm được thiết kế API nhất quán cho các công cụ
3. Hiểu được cách tích hợp và sử dụng các công cụ trong quy trình tự động
4. Có thể mở rộng thư viện với các công cụ mới

## Trích Xuất Từ Bài Báo

> "The foundation of our solution is a curated library of data science tools, encompassing functions for data cleaning, feature engineering, and modeling. This library not only provides the necessary tools for data processing and analysis but also ensures that the generated code is reliable and follows best practices." (Section 3.3 Machine Learning Tools Library, Page 6)

> "The ML tools library is organized into three main categories: data cleaning tools, feature engineering tools, and model building, validation, and prediction tools. Each category contains a set of functions that are commonly used in data science competitions and have been validated through extensive testing." (Section 3.3 Machine Learning Tools Library, Page 6)

> "Data Cleaning Tools: This category includes functions for handling missing values, removing duplicates, detecting and handling outliers, and converting data types. These tools are essential for preparing clean, high-quality data for analysis and modeling." (Section 3.3 Machine Learning Tools Library, Page 6)

> "Feature Engineering Tools: This category encompasses functions for creating new features, transforming existing features, encoding categorical variables, and feature selection. These tools help improve model performance by creating informative and relevant features." (Section 3.3 Machine Learning Tools Library, Page 6)

> "Model Building, Validation, and Prediction Tools: This category includes functions for training various machine learning models, evaluating model performance, performing cross-validation, and making predictions. These tools support the entire modeling workflow from training to final prediction." (Section 3.3 Machine Learning Tools Library, Page 6)

![ML Tools Library Structure](https://m-a-p.ai/AutoKaggle.github.io/assets/images/fig3.png)
*(Note: Diagram representation - actual image not included)*

## Cài Đặt Môi Trường

In [ ]:
!pip install -q pandas numpy scikit-learn matplotlib seaborn
!pip install -q xgboost lightgbm catboost
!pip install -q category_encoders feature-engine
!pip install -q optuna bayesian-optimization

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Optional, Tuple, Any, Union, Callable
from abc import ABC, abstractmethod
import warnings
warnings.filterwarnings('ignore')

# Import machine learning libraries
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.feature_selection import SelectKBest, f_classif, f_regression

# Import advanced ML libraries
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier, CatBoostRegressor

## Phần 1: Thiết Kế Kiến Trúc Thư Viện Công Cụ

Theo bài báo, thư viện công cụ ML được tổ chức thành 3 danh mục chính. Chúng ta sẽ thiết kế một kiến trúc có thể mở rộng và dễ bảo trì:

In [ ]:
# Base class cho tất cả các công cụ ML
class MLTool(ABC):
    """
    Abstract base class cho tất cả các công cụ ML trong thư viện
    
    Mỗi công cụ phải implement các phương thức cơ bản:
    - execute: Thực thi chức năng chính của công cụ
    - validate_input: Kiểm tra tính hợp lệ của đầu vào
    - get_description: Trả về mô tả chi tiết về công cụ
    """
    
    def __init__(self, name: str, category: str, description: str):
        self.name = name
        self.category = category
        self.description = description
        self.execution_log = []
    
    @abstractmethod
    def execute(self, *args, **kwargs) -> Any:
        """Thực thi chức năng chính của công cụ"""
        pass
    
    @abstractmethod
    def validate_input(self, *args, **kwargs) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        pass
    
    def get_description(self) -> str:
        """Trả về mô tả chi tiết về công cụ"""
        return f"{self.name} ({self.category}): {self.description}"
    
    def log_execution(self, inputs: Dict, outputs: Dict, execution_time: float):
        """Ghi log quá trình thực thi"""
        log_entry = {
            'timestamp': pd.Timestamp.now(),
            'inputs': inputs,
            'outputs': outputs,
            'execution_time': execution_time
        }
        self.execution_log.append(log_entry)

# Enum cho các danh mục công cụ
class ToolCategory:
    DATA_CLEANING = "Data Cleaning"
    FEATURE_ENGINEERING = "Feature Engineering"
    MODEL_BUILDING = "Model Building, Validation & Prediction"

# Registry để quản lý tất cả các công cụ
class MLToolRegistry:
    """
    Registry pattern để quản lý và truy cập các công cụ ML
    """
    
    def __init__(self):
        self._tools = {}
        self._categories = {}
    
    def register_tool(self, tool: MLTool):
        """Đăng ký một công cụ mới"""
        self._tools[tool.name] = tool
        
        if tool.category not in self._categories:
            self._categories[tool.category] = []
        self._categories[tool.category].append(tool.name)
    
    def get_tool(self, name: str) -> Optional[MLTool]:
        """Lấy công cụ theo tên"""
        return self._tools.get(name)
    
    def get_tools_by_category(self, category: str) -> List[MLTool]:
        """Lấy tất cả công cụ trong một danh mục"""
        tool_names = self._categories.get(category, [])
        return [self._tools[name] for name in tool_names]
    
    def list_all_tools(self) -> Dict[str, List[str]]:
        """Liệt kê tất cả công cụ theo danh mục"""
        return self._categories.copy()
    
    def search_tools(self, keyword: str) -> List[MLTool]:
        """Tìm kiếm công cụ theo từ khóa"""
        results = []
        for tool in self._tools.values():
            if (keyword.lower() in tool.name.lower() or 
                keyword.lower() in tool.description.lower()):
                results.append(tool)
        return results

# Khởi tạo registry toàn cục
ml_registry = MLToolRegistry()

## Phần 2: Danh Mục 1 - Công Cụ Làm Sạch Dữ Liệu (Data Cleaning Tools)

Danh mục này bao gồm các công cụ để xử lý dữ liệu thiếu, loại bỏ bản sao, phát hiện và xử lý dữ liệu ngoại lai, và chuyển đổi kiểu dữ liệu.

In [ ]:
import time
from scipy import stats

class MissingValueHandler(MLTool):
    """
    Công cụ xử lý giá trị thiếu với nhiều chiến lược khác nhau
    """
    
    def __init__(self):
        super().__init__(
            name="missing_value_handler",
            category=ToolCategory.DATA_CLEANING,
            description="Handle missing values using various strategies (mean, median, mode, forward fill, etc.)"
        )
        self.supported_strategies = {
            'mean': self._fill_mean,
            'median': self._fill_median,
            'mode': self._fill_mode,
            'forward_fill': self._forward_fill,
            'backward_fill': self._backward_fill,
            'interpolate': self._interpolate,
            'drop': self._drop_missing,
            'constant': self._fill_constant
        }
    
    def validate_input(self, df: pd.DataFrame, strategy: Dict[str, str]) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if not isinstance(df, pd.DataFrame):
            return False
        if not isinstance(strategy, dict):
            return False
        
        # Kiểm tra xem các cột có tồn tại không
        for col in strategy.keys():
            if col not in df.columns:
                return False
        
        # Kiểm tra xem các chiến lược có được hỗ trợ không
        for strat in strategy.values():
            if isinstance(strat, str) and strat not in self.supported_strategies:
                return False
        
        return True
    
    def execute(self, df: pd.DataFrame, strategy: Dict[str, Union[str, Any]]) -> pd.DataFrame:
        """
        Xử lý giá trị thiếu theo chiến lược được chỉ định
        
        Parameters:
        - df: DataFrame cần xử lý
        - strategy: Dictionary với key là tên cột, value là chiến lược xử lý
        
        Returns:
        - DataFrame đã xử lý giá trị thiếu
        """
        start_time = time.time()
        
        if not self.validate_input(df, strategy):
            raise ValueError("Invalid input parameters")
        
        df_processed = df.copy()
        missing_info = {}
        
        for column, method in strategy.items():
            if column not in df_processed.columns:
                continue
            
            # Ghi nhận thông tin về giá trị thiếu trước khi xử lý
            missing_count = df_processed[column].isnull().sum()
            missing_info[column] = {
                'missing_count_before': missing_count,
                'missing_percentage_before': (missing_count / len(df_processed)) * 100,
                'strategy_used': method
            }
            
            if missing_count == 0:
                continue
            
            # Áp dụng chiến lược xử lý
            if isinstance(method, str) and method in self.supported_strategies:
                df_processed = self.supported_strategies[method](df_processed, column)
            else:
                # Sử dụng giá trị constant
                df_processed[column].fillna(method, inplace=True)
            
            # Ghi nhận thông tin sau khi xử lý
            missing_count_after = df_processed[column].isnull().sum()
            missing_info[column]['missing_count_after'] = missing_count_after
            missing_info[column]['missing_percentage_after'] = (missing_count_after / len(df_processed)) * 100
        
        execution_time = time.time() - start_time
        
        # Log thực thi
        self.log_execution(
            inputs={'dataframe_shape': df.shape, 'strategy': strategy},
            outputs={'processed_shape': df_processed.shape, 'missing_info': missing_info},
            execution_time=execution_time
        )
        
        return df_processed
    
    def _fill_mean(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Điền giá trị trung bình"""
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column].fillna(df[column].mean(), inplace=True)
        return df
    
    def _fill_median(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Điền giá trị trung vị"""
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column].fillna(df[column].median(), inplace=True)
        return df
    
    def _fill_mode(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Điền giá trị mode (phổ biến nhất)"""
        mode_value = df[column].mode()
        if not mode_value.empty:
            df[column].fillna(mode_value[0], inplace=True)
        return df
    
    def _forward_fill(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Điền giá trị phía trước"""
        df[column].fillna(method='ffill', inplace=True)
        return df
    
    def _backward_fill(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Điền giá trị phía sau"""
        df[column].fillna(method='bfill', inplace=True)
        return df
    
    def _interpolate(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Nội suy giá trị"""
        if pd.api.types.is_numeric_dtype(df[column]):
            df[column].interpolate(inplace=True)
        return df
    
    def _drop_missing(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Loại bỏ hàng có giá trị thiếu"""
        return df.dropna(subset=[column])
    
    def _fill_constant(self, df: pd.DataFrame, column: str, value: Any = 0) -> pd.DataFrame:
        """Điền giá trị constant"""
        df[column].fillna(value, inplace=True)
        return df

class OutlierDetector(MLTool):
    """
    Công cụ phát hiện và xử lý dữ liệu ngoại lai
    """
    
    def __init__(self):
        super().__init__(
            name="outlier_detector",
            category=ToolCategory.DATA_CLEANING,
            description="Detect and handle outliers using IQR, Z-score, and Isolation Forest methods"
        )
        self.supported_methods = ['iqr', 'zscore', 'isolation_forest']
        self.supported_actions = ['remove', 'cap', 'transform']
    
    def validate_input(self, df: pd.DataFrame, columns: List[str], method: str, action: str) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if not isinstance(df, pd.DataFrame):
            return False
        if not all(col in df.columns for col in columns):
            return False
        if method not in self.supported_methods:
            return False
        if action not in self.supported_actions:
            return False
        return True
    
    def execute(self, df: pd.DataFrame, columns: List[str], 
                method: str = 'iqr', action: str = 'cap', 
                threshold: float = 1.5) -> Tuple[pd.DataFrame, Dict]:
        """
        Phát hiện và xử lý dữ liệu ngoại lai
        
        Parameters:
        - df: DataFrame cần xử lý
        - columns: Danh sách cột cần kiểm tra
        - method: Phương pháp phát hiện ('iqr', 'zscore', 'isolation_forest')
        - action: Hành động xử lý ('remove', 'cap', 'transform')
        - threshold: Ngưỡng phát hiện
        
        Returns:
        - Tuple của (DataFrame đã xử lý, thông tin outlier)
        """
        start_time = time.time()
        
        if not self.validate_input(df, columns, method, action):
            raise ValueError("Invalid input parameters")
        
        df_processed = df.copy()
        outlier_info = {}
        
        for column in columns:
            if not pd.api.types.is_numeric_dtype(df_processed[column]):
                continue
            
            # Phát hiện outlier
            if method == 'iqr':
                outlier_mask = self._detect_outliers_iqr(df_processed[column], threshold)
            elif method == 'zscore':
                outlier_mask = self._detect_outliers_zscore(df_processed[column], threshold)
            elif method == 'isolation_forest':
                outlier_mask = self._detect_outliers_isolation_forest(df_processed[column])
            
            outlier_count = outlier_mask.sum()
            outlier_info[column] = {
                'outlier_count': outlier_count,
                'outlier_percentage': (outlier_count / len(df_processed)) * 100,
                'method_used': method,
                'action_taken': action
            }
            
            # Xử lý outlier
            if action == 'remove':
                df_processed = df_processed[~outlier_mask]
            elif action == 'cap':
                df_processed = self._cap_outliers(df_processed, column, method, threshold)
            elif action == 'transform':
                df_processed[column] = self._transform_outliers(df_processed[column])
        
        execution_time = time.time() - start_time
        
        # Log thực thi
        self.log_execution(
            inputs={'dataframe_shape': df.shape, 'columns': columns, 'method': method, 'action': action},
            outputs={'processed_shape': df_processed.shape, 'outlier_info': outlier_info},
            execution_time=execution_time
        )
        
        return df_processed, outlier_info
    
    def _detect_outliers_iqr(self, series: pd.Series, threshold: float = 1.5) -> pd.Series:
        """Phát hiện outlier bằng phương pháp IQR"""
        Q1 = series.quantile(0.25)
        Q3 = series.quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - threshold * IQR
        upper_bound = Q3 + threshold * IQR
        return (series < lower_bound) | (series > upper_bound)
    
    def _detect_outliers_zscore(self, series: pd.Series, threshold: float = 3.0) -> pd.Series:
        """Phát hiện outlier bằng phương pháp Z-score"""
        z_scores = np.abs(stats.zscore(series.dropna()))
        outlier_mask = pd.Series(False, index=series.index)
        outlier_mask[series.dropna().index] = z_scores > threshold
        return outlier_mask
    
    def _detect_outliers_isolation_forest(self, series: pd.Series) -> pd.Series:
        """Phát hiện outlier bằng Isolation Forest"""
        from sklearn.ensemble import IsolationForest
        
        # Chuẩn bị dữ liệu
        data = series.dropna().values.reshape(-1, 1)
        
        # Khởi tạo và fit model
        iso_forest = IsolationForest(contamination=0.1, random_state=42)
        outlier_labels = iso_forest.fit_predict(data)
        
        # Tạo mask outlier
        outlier_mask = pd.Series(False, index=series.index)
        outlier_mask[series.dropna().index] = outlier_labels == -1
        
        return outlier_mask
    
    def _cap_outliers(self, df: pd.DataFrame, column: str, method: str, threshold: float) -> pd.DataFrame:
        """Giới hạn giá trị outlier trong khoảng cho phép"""
        if method == 'iqr':
            Q1 = df[column].quantile(0.25)
            Q3 = df[column].quantile(0.75)
            IQR = Q3 - Q1
            lower_bound = Q1 - threshold * IQR
            upper_bound = Q3 + threshold * IQR
        elif method == 'zscore':
            mean = df[column].mean()
            std = df[column].std()
            lower_bound = mean - threshold * std
            upper_bound = mean + threshold * std
        
        df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
        return df
    
    def _transform_outliers(self, series: pd.Series) -> pd.Series:
        """Biến đổi outlier bằng log transformation"""
        # Áp dụng log transformation (thêm 1 để tránh log(0))
        return np.log1p(np.abs(series))

# Đăng ký các công cụ data cleaning
ml_registry.register_tool(MissingValueHandler())
ml_registry.register_tool(OutlierDetector())

## Phần 3: Danh Mục 2 - Công Cụ Kỹ Thuật Đặc Trưng (Feature Engineering Tools)

Danh mục này bao gồm các công cụ để tạo đặc trưng mới, biến đổi đặc trưng hiện có, mã hóa biến phân loại, và lựa chọn đặc trưng.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif, mutual_info_regression
from sklearn.decomposition import PCA
from category_encoders import TargetEncoder, BinaryEncoder, CatBoostEncoder

class CategoricalEncoder(MLTool):
    """
    Công cụ mã hóa biến phân loại với nhiều phương pháp khác nhau
    """
    
    def __init__(self):
        super().__init__(
            name="categorical_encoder",
            category=ToolCategory.FEATURE_ENGINEERING,
            description="Encode categorical variables using various methods (one-hot, label, target, binary encoding)"
        )
        self.supported_methods = {
            'onehot': self._onehot_encode,
            'label': self._label_encode,
            'target': self._target_encode,
            'binary': self._binary_encode,
            'frequency': self._frequency_encode,
            'ordinal': self._ordinal_encode
        }
        self.encoders = {}
    
    def validate_input(self, df: pd.DataFrame, columns: List[str], method: str) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if not isinstance(df, pd.DataFrame):
            return False
        if not all(col in df.columns for col in columns):
            return False
        if method not in self.supported_methods:
            return False
        return True
    
    def execute(self, df: pd.DataFrame, columns: List[str], method: str = 'onehot', 
                target: Optional[str] = None, mapping: Optional[Dict] = None) -> pd.DataFrame:
        """
        Mã hóa biến phân loại
        
        Parameters:
        - df: DataFrame cần xử lý
        - columns: Danh sách cột phân loại cần mã hóa
        - method: Phương pháp mã hóa
        - target: Cột target (cần thiết cho target encoding)
        - mapping: Mapping cho ordinal encoding
        
        Returns:
        - DataFrame đã mã hóa
        """
        start_time = time.time()
        
        if not self.validate_input(df, columns, method):
            raise ValueError("Invalid input parameters")
        
        df_processed = df.copy()
        encoding_info = {}
        
        for column in columns:
            if column not in df_processed.columns:
                continue
            
            original_unique_count = df_processed[column].nunique()
            
            # Áp dụng phương pháp mã hóa
            if method == 'target' and target:
                df_processed = self.supported_methods[method](df_processed, column, target)
            elif method == 'ordinal' and mapping:
                df_processed = self.supported_methods[method](df_processed, column, mapping.get(column, {}))
            else:
                df_processed = self.supported_methods[method](df_processed, column)
            
            # Ghi nhận thông tin encoding
            if method == 'onehot':
                new_columns = [col for col in df_processed.columns if col.startswith(f"{column}_")]
                encoding_info[column] = {
                    'method': method,
                    'original_unique_count': original_unique_count,
                    'new_columns_count': len(new_columns),
                    'new_columns': new_columns
                }
            else:
                encoding_info[column] = {
                    'method': method,
                    'original_unique_count': original_unique_count,
                    'new_columns_count': 1
                }
        
        execution_time = time.time() - start_time
        
        # Log thực thi
        self.log_execution(
            inputs={'dataframe_shape': df.shape, 'columns': columns, 'method': method},
            outputs={'processed_shape': df_processed.shape, 'encoding_info': encoding_info},
            execution_time=execution_time
        )
        
        return df_processed
    
    def _onehot_encode(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """One-hot encoding"""
        dummies = pd.get_dummies(df[column], prefix=column, dummy_na=False)
        df = pd.concat([df.drop(column, axis=1), dummies], axis=1)
        return df
    
    def _label_encode(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Label encoding"""
        le = LabelEncoder()
        df[column] = le.fit_transform(df[column].astype(str))
        self.encoders[f"{column}_label"] = le
        return df
    
    def _target_encode(self, df: pd.DataFrame, column: str, target: str) -> pd.DataFrame:
        """Target encoding"""
        te = TargetEncoder()
        df[column] = te.fit_transform(df[column], df[target])
        self.encoders[f"{column}_target"] = te
        return df
    
    def _binary_encode(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Binary encoding"""
        be = BinaryEncoder(cols=[column])
        encoded_df = be.fit_transform(df[[column]])
        df = pd.concat([df.drop(column, axis=1), encoded_df], axis=1)
        self.encoders[f"{column}_binary"] = be
        return df
    
    def _frequency_encode(self, df: pd.DataFrame, column: str) -> pd.DataFrame:
        """Frequency encoding"""
        freq_map = df[column].value_counts(normalize=True).to_dict()
        df[f"{column}_freq"] = df[column].map(freq_map)
        df.drop(column, axis=1, inplace=True)
        self.encoders[f"{column}_frequency"] = freq_map
        return df
    
    def _ordinal_encode(self, df: pd.DataFrame, column: str, mapping: Dict) -> pd.DataFrame:
        """Ordinal encoding với mapping tùy chỉnh"""
        df[column] = df[column].map(mapping)
        return df

class FeatureCreator(MLTool):
    """
    Công cụ tạo đặc trưng mới từ các đặc trưng hiện có
    """
    
    def __init__(self):
        super().__init__(
            name="feature_creator",
            category=ToolCategory.FEATURE_ENGINEERING,
            description="Create new features through mathematical operations, polynomial features, and interactions"
        )
        self.supported_operations = {
            'polynomial': self._create_polynomial_features,
            'interaction': self._create_interaction_features, 
            'mathematical': self._create_mathematical_features,
            'binning': self._create_binned_features,
            'datetime': self._create_datetime_features
        }
    
    def validate_input(self, df: pd.DataFrame, operation: str) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if not isinstance(df, pd.DataFrame):
            return False
        if operation not in self.supported_operations:
            return False
        return True
    
    def execute(self, df: pd.DataFrame, operation: str, 
                columns: Optional[List[str]] = None, 
                degree: int = 2, 
                bins: int = 5,
                math_operations: List[str] = ['log', 'sqrt', 'square']) -> pd.DataFrame:
        """
        Tạo đặc trưng mới
        
        Parameters:
        - df: DataFrame cần xử lý
        - operation: Loại operation ('polynomial', 'interaction', 'mathematical', 'binning', 'datetime')
        - columns: Danh sách cột để áp dụng (None để áp dụng cho tất cả cột số)
        - degree: Bậc cho polynomial features
        - bins: Số bins cho binning
        - math_operations: Danh sách phép toán số học
        
        Returns:
        - DataFrame với các đặc trưng mới
        """
        start_time = time.time()
        
        if not self.validate_input(df, operation):
            raise ValueError("Invalid input parameters")
        
        df_processed = df.copy()
        
        # Xác định cột để xử lý
        if columns is None:
            if operation == 'datetime':
                columns = df_processed.select_dtypes(include=['datetime64', 'object']).columns.tolist()
            else:
                columns = df_processed.select_dtypes(include=[np.number]).columns.tolist()
        
        original_columns = df_processed.shape[1]
        
        # Áp dụng operation
        if operation == 'polynomial':
            df_processed = self.supported_operations[operation](df_processed, columns, degree)
        elif operation == 'binning':
            df_processed = self.supported_operations[operation](df_processed, columns, bins)
        elif operation == 'mathematical':
            df_processed = self.supported_operations[operation](df_processed, columns, math_operations)
        else:
            df_processed = self.supported_operations[operation](df_processed, columns)
        
        new_columns = df_processed.shape[1]
        features_created = new_columns - original_columns
        
        execution_time = time.time() - start_time
        
        # Log thực thi
        self.log_execution(
            inputs={'dataframe_shape': df.shape, 'operation': operation, 'columns': columns},
            outputs={'processed_shape': df_processed.shape, 'features_created': features_created},
            execution_time=execution_time
        )
        
        return df_processed
    
    def _create_polynomial_features(self, df: pd.DataFrame, columns: List[str], degree: int) -> pd.DataFrame:
        """Tạo polynomial features"""
        poly = PolynomialFeatures(degree=degree, include_bias=False)
        
        # Chỉ áp dụng cho cột số
        numeric_columns = [col for col in columns if pd.api.types.is_numeric_dtype(df[col])]
        
        if numeric_columns:
            poly_features = poly.fit_transform(df[numeric_columns])
            poly_feature_names = poly.get_feature_names_out(numeric_columns)
            
            # Loại bỏ các features gốc (đã có sẵn)
            new_features = poly_features[:, len(numeric_columns):]
            new_feature_names = poly_feature_names[len(numeric_columns):]
            
            # Thêm vào DataFrame
            for i, name in enumerate(new_feature_names):
                df[f"poly_{name}"] = new_features[:, i]
        
        return df
    
    def _create_interaction_features(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        """Tạo interaction features"""
        numeric_columns = [col for col in columns if pd.api.types.is_numeric_dtype(df[col])]
        
        # Tạo tất cả các cặp tương tác
        for i in range(len(numeric_columns)):
            for j in range(i+1, len(numeric_columns)):
                col1, col2 = numeric_columns[i], numeric_columns[j]
                df[f"{col1}_x_{col2}"] = df[col1] * df[col2]
                df[f"{col1}_div_{col2}"] = df[col1] / (df[col2] + 1e-8)  # Tránh chia cho 0
        
        return df
    
    def _create_mathematical_features(self, df: pd.DataFrame, columns: List[str], operations: List[str]) -> pd.DataFrame:
        """Tạo mathematical features"""
        numeric_columns = [col for col in columns if pd.api.types.is_numeric_dtype(df[col])]
        
        for col in numeric_columns:
            if 'log' in operations:
                df[f"{col}_log"] = np.log1p(np.abs(df[col]))  # log(1+x) để tránh log(0)
            if 'sqrt' in operations:
                df[f"{col}_sqrt"] = np.sqrt(np.abs(df[col]))
            if 'square' in operations:
                df[f"{col}_square"] = df[col] ** 2
            if 'reciprocal' in operations:
                df[f"{col}_reciprocal"] = 1 / (df[col] + 1e-8)
        
        return df
    
    def _create_binned_features(self, df: pd.DataFrame, columns: List[str], bins: int) -> pd.DataFrame:
        """Tạo binned features"""
        numeric_columns = [col for col in columns if pd.api.types.is_numeric_dtype(df[col])]
        
        for col in numeric_columns:
            df[f"{col}_binned"] = pd.cut(df[col], bins=bins, labels=False)
        
        return df
    
    def _create_datetime_features(self, df: pd.DataFrame, columns: List[str]) -> pd.DataFrame:
        """Tạo datetime features"""
        for col in columns:
            # Thử convert sang datetime nếu chưa phải
            if not pd.api.types.is_datetime64_any_dtype(df[col]):
                try:
                    df[col] = pd.to_datetime(df[col])
                except:
                    continue
            
            # Tạo các features từ datetime
            df[f"{col}_year"] = df[col].dt.year
            df[f"{col}_month"] = df[col].dt.month
            df[f"{col}_day"] = df[col].dt.day
            df[f"{col}_dayofweek"] = df[col].dt.dayofweek
            df[f"{col}_hour"] = df[col].dt.hour
            df[f"{col}_is_weekend"] = (df[col].dt.dayofweek >= 5).astype(int)
        
        return df

# Đăng ký các công cụ feature engineering
ml_registry.register_tool(CategoricalEncoder())
ml_registry.register_tool(FeatureCreator())

## Phần 4: Danh Mục 3 - Công Cụ Xây Dựng, Xác Thực và Dự Đoán Mô Hình

Danh mục này bao gồm các công cụ để huấn luyện mô hình, đánh giá hiệu suất, thực hiện cross-validation, và tạo dự đoán.

In [ ]:
from sklearn.model_selection import cross_val_score, StratifiedKFold, KFold
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
import joblib

class ModelTrainer(MLTool):
    """
    Công cụ huấn luyện và so sánh nhiều mô hình học máy
    """
    
    def __init__(self):
        super().__init__(
            name="model_trainer",
            category=ToolCategory.MODEL_BUILDING,
            description="Train and compare multiple machine learning models with automatic hyperparameter tuning"
        )
        self.supported_models = {
            # Classification models
            'random_forest_clf': RandomForestClassifier,
            'gradient_boosting_clf': GradientBoostingClassifier,
            'logistic_regression': LogisticRegression,
            'svm_clf': SVC,
            'knn_clf': KNeighborsClassifier,
            'naive_bayes': GaussianNB,
            'decision_tree_clf': DecisionTreeClassifier,
            'xgboost_clf': xgb.XGBClassifier,
            'lightgbm_clf': lgb.LGBMClassifier,
            
            # Regression models
            'random_forest_reg': RandomForestRegressor,
            'gradient_boosting_reg': GradientBoostingRegressor,
            'ridge': Ridge,
            'lasso': Lasso,
            'svm_reg': SVR,
            'xgboost_reg': xgb.XGBRegressor,
            'lightgbm_reg': lgb.LGBMRegressor
        }
        self.trained_models = {}
    
    def validate_input(self, X: pd.DataFrame, y: pd.Series, problem_type: str) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if not isinstance(X, pd.DataFrame) or not isinstance(y, pd.Series):
            return False
        if len(X) != len(y):
            return False
        if problem_type not in ['classification', 'regression']:
            return False
        return True
    
    def execute(self, X: pd.DataFrame, y: pd.Series, 
                problem_type: str = 'classification',
                models_to_try: Optional[List[str]] = None,
                cv_folds: int = 5,
                scoring: Optional[str] = None,
                test_size: float = 0.2,
                random_state: int = 42) -> Dict[str, Any]:
        """
        Huấn luyện và so sánh nhiều mô hình
        
        Parameters:
        - X: Features
        - y: Target variable
        - problem_type: 'classification' hoặc 'regression'
        - models_to_try: Danh sách mô hình muốn thử (None để thử tất cả)
        - cv_folds: Số fold cho cross-validation
        - scoring: Metric đánh giá
        - test_size: Tỷ lệ test set
        - random_state: Random seed
        
        Returns:
        - Dictionary chứa kết quả huấn luyện và so sánh
        """
        start_time = time.time()
        
        if not self.validate_input(X, y, problem_type):
            raise ValueError("Invalid input parameters")
        
        # Xác định models và scoring mặc định
        if models_to_try is None:
            if problem_type == 'classification':
                models_to_try = ['random_forest_clf', 'gradient_boosting_clf', 'logistic_regression', 'xgboost_clf']
            else:
                models_to_try = ['random_forest_reg', 'gradient_boosting_reg', 'ridge', 'xgboost_reg']
        
        if scoring is None:
            scoring = 'accuracy' if problem_type == 'classification' else 'neg_mean_squared_error'
        
        # Chia dữ liệu
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=test_size, random_state=random_state, 
            stratify=y if problem_type == 'classification' else None
        )
        
        # Thiết lập cross-validation
        if problem_type == 'classification':
            cv = StratifiedKFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
        else:
            cv = KFold(n_splits=cv_folds, shuffle=True, random_state=random_state)
        
        # Huấn luyện và đánh giá các mô hình
        results = {}
        model_performances = []
        
        for model_name in models_to_try:
            if model_name not in self.supported_models:
                continue
            
            try:
                # Khởi tạo mô hình với tham số mặc định
                model_class = self.supported_models[model_name]
                model = model_class(random_state=random_state)
                
                # Cross-validation
                cv_scores = cross_val_score(model, X_train, y_train, cv=cv, scoring=scoring)
                
                # Huấn luyện trên toàn bộ training set
                model.fit(X_train, y_train)
                
                # Đánh giá trên test set
                test_score = model.score(X_test, y_test)
                
                # Lưu kết quả
                model_result = {
                    'model': model,
                    'cv_mean': cv_scores.mean(),
                    'cv_std': cv_scores.std(),
                    'test_score': test_score,
                    'cv_scores': cv_scores
                }
                
                results[model_name] = model_result
                self.trained_models[model_name] = model
                
                model_performances.append({
                    'model_name': model_name,
                    'cv_mean': cv_scores.mean(),
                    'cv_std': cv_scores.std(),
                    'test_score': test_score
                })
                
            except Exception as e:
                print(f"Error training {model_name}: {e}")
                continue
        
        # Tìm mô hình tốt nhất
        if model_performances:
            best_model_info = max(model_performances, key=lambda x: x['cv_mean'])
            best_model_name = best_model_info['model_name']
            best_model = results[best_model_name]['model']
        else:
            best_model_info = None
            best_model_name = None
            best_model = None
        
        execution_time = time.time() - start_time
        
        # Tạo DataFrame so sánh
        comparison_df = pd.DataFrame(model_performances)
        if not comparison_df.empty:
            comparison_df = comparison_df.sort_values('cv_mean', ascending=False)
        
        final_results = {
            'model_results': results,
            'comparison_df': comparison_df,
            'best_model': best_model,
            'best_model_name': best_model_name,
            'best_model_info': best_model_info,
            'X_train': X_train,
            'X_test': X_test,
            'y_train': y_train,
            'y_test': y_test
        }
        
        # Log thực thi
        self.log_execution(
            inputs={'X_shape': X.shape, 'y_shape': y.shape, 'problem_type': problem_type, 'models_to_try': models_to_try},
            outputs={'models_trained': len(results), 'best_model': best_model_name},
            execution_time=execution_time
        )
        
        return final_results
    
    def save_model(self, model_name: str, filepath: str):
        """Lưu mô hình đã huấn luyện"""
        if model_name in self.trained_models:
            joblib.dump(self.trained_models[model_name], filepath)
        else:
            raise ValueError(f"Model {model_name} not found in trained models")
    
    def load_model(self, filepath: str, model_name: str):
        """Tải mô hình đã lưu"""
        model = joblib.load(filepath)
        self.trained_models[model_name] = model
        return model

class ModelEvaluator(MLTool):
    """
    Công cụ đánh giá mô hình với nhiều metrics khác nhau
    """
    
    def __init__(self):
        super().__init__(
            name="model_evaluator",
            category=ToolCategory.MODEL_BUILDING,
            description="Evaluate model performance using various metrics and generate comprehensive reports"
        )
    
    def validate_input(self, y_true: pd.Series, y_pred: np.ndarray, problem_type: str) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if len(y_true) != len(y_pred):
            return False
        if problem_type not in ['classification', 'regression']:
            return False
        return True
    
    def execute(self, y_true: pd.Series, y_pred: np.ndarray, 
                problem_type: str = 'classification',
                y_pred_proba: Optional[np.ndarray] = None) -> Dict[str, Any]:
        """
        Đánh giá hiệu suất mô hình
        
        Parameters:
        - y_true: Giá trị thực tế
        - y_pred: Giá trị dự đoán
        - problem_type: 'classification' hoặc 'regression'
        - y_pred_proba: Xác suất dự đoán (cho classification)
        
        Returns:
        - Dictionary chứa các metrics đánh giá
        """
        start_time = time.time()
        
        if not self.validate_input(y_true, y_pred, problem_type):
            raise ValueError("Invalid input parameters")
        
        metrics = {}
        
        if problem_type == 'classification':
            # Classification metrics
            metrics['accuracy'] = accuracy_score(y_true, y_pred)
            metrics['precision'] = precision_score(y_true, y_pred, average='weighted')
            metrics['recall'] = recall_score(y_true, y_pred, average='weighted')
            metrics['f1_score'] = f1_score(y_true, y_pred, average='weighted')
            
            # Classification report
            from sklearn.metrics import classification_report, confusion_matrix
            metrics['classification_report'] = classification_report(y_true, y_pred)
            metrics['confusion_matrix'] = confusion_matrix(y_true, y_pred)
            
            # AUC-ROC nếu có probability
            if y_pred_proba is not None:
                from sklearn.metrics import roc_auc_score
                try:
                    if len(np.unique(y_true)) == 2:  # Binary classification
                        metrics['auc_roc'] = roc_auc_score(y_true, y_pred_proba[:, 1])
                    else:  # Multi-class
                        metrics['auc_roc'] = roc_auc_score(y_true, y_pred_proba, multi_class='ovr')
                except Exception as e:
                    metrics['auc_roc'] = f"Cannot compute AUC-ROC: {e}"
        
        else:  # Regression
            # Regression metrics
            metrics['mse'] = mean_squared_error(y_true, y_pred)
            metrics['rmse'] = np.sqrt(metrics['mse'])
            metrics['mae'] = mean_absolute_error(y_true, y_pred)
            metrics['r2_score'] = r2_score(y_true, y_pred)
            
            # Additional regression metrics
            from sklearn.metrics import mean_absolute_percentage_error
            try:
                metrics['mape'] = mean_absolute_percentage_error(y_true, y_pred)
            except:
                metrics['mape'] = "Cannot compute MAPE (possible zero values)"
        
        execution_time = time.time() - start_time
        
        # Log thực thi
        self.log_execution(
            inputs={'y_true_shape': y_true.shape, 'y_pred_shape': y_pred.shape, 'problem_type': problem_type},
            outputs={'metrics_computed': list(metrics.keys())},
            execution_time=execution_time
        )
        
        return metrics
    
    def plot_evaluation_results(self, y_true: pd.Series, y_pred: np.ndarray, 
                               problem_type: str = 'classification',
                               y_pred_proba: Optional[np.ndarray] = None):
        """Vẽ biểu đồ đánh giá kết quả"""
        if problem_type == 'classification':
            from sklearn.metrics import confusion_matrix
            import seaborn as sns
            
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            
            # Confusion Matrix
            cm = confusion_matrix(y_true, y_pred)
            sns.heatmap(cm, annot=True, fmt='d', ax=axes[0])
            axes[0].set_title('Confusion Matrix')
            axes[0].set_xlabel('Predicted')
            axes[0].set_ylabel('Actual')
            
            # ROC Curve (nếu có probability)
            if y_pred_proba is not None and len(np.unique(y_true)) == 2:
                from sklearn.metrics import roc_curve, auc
                fpr, tpr, _ = roc_curve(y_true, y_pred_proba[:, 1])
                roc_auc = auc(fpr, tpr)
                
                axes[1].plot(fpr, tpr, label=f'ROC curve (AUC = {roc_auc:.2f})')
                axes[1].plot([0, 1], [0, 1], 'k--')
                axes[1].set_xlim([0.0, 1.0])
                axes[1].set_ylim([0.0, 1.05])
                axes[1].set_xlabel('False Positive Rate')
                axes[1].set_ylabel('True Positive Rate')
                axes[1].set_title('ROC Curve')
                axes[1].legend()
            else:
                axes[1].text(0.5, 0.5, 'ROC Curve\nNot Available', 
                           ha='center', va='center', transform=axes[1].transAxes)
        
        else:  # Regression
            fig, axes = plt.subplots(1, 2, figsize=(12, 5))
            
            # Actual vs Predicted
            axes[0].scatter(y_true, y_pred, alpha=0.6)
            axes[0].plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'r--', lw=2)
            axes[0].set_xlabel('Actual')
            axes[0].set_ylabel('Predicted')
            axes[0].set_title('Actual vs Predicted')
            
            # Residuals
            residuals = y_true - y_pred
            axes[1].scatter(y_pred, residuals, alpha=0.6)
            axes[1].axhline(y=0, color='r', linestyle='--')
            axes[1].set_xlabel('Predicted')
            axes[1].set_ylabel('Residuals')
            axes[1].set_title('Residual Plot')
        
        plt.tight_layout()
        return fig

# Đăng ký các công cụ model building
ml_registry.register_tool(ModelTrainer())
ml_registry.register_tool(ModelEvaluator())

## Phần 5: Quản Lý và Sử Dụng Thư Viện Công Cụ

Bây giờ chúng ta sẽ tạo một interface để quản lý và sử dụng toàn bộ thư viện công cụ một cách hiệu quả:

In [ ]:
class MLToolsManager:
    """
    Manager class để quản lý và sử dụng toàn bộ thư viện công cụ ML
    """
    
    def __init__(self, registry: MLToolRegistry):
        self.registry = registry
        self.workflow_history = []
    
    def list_available_tools(self) -> pd.DataFrame:
        """Liệt kê tất cả công cụ có sẵn"""
        tools_info = []
        
        for category, tool_names in self.registry.list_all_tools().items():
            for tool_name in tool_names:
                tool = self.registry.get_tool(tool_name)
                tools_info.append({
                    'Tool Name': tool.name,
                    'Category': tool.category,
                    'Description': tool.description
                })
        
        return pd.DataFrame(tools_info)
    
    def search_tools(self, keyword: str) -> pd.DataFrame:
        """Tìm kiếm công cụ theo từ khóa"""
        tools = self.registry.search_tools(keyword)
        tools_info = []
        
        for tool in tools:
            tools_info.append({
                'Tool Name': tool.name,
                'Category': tool.category,
                'Description': tool.description
            })
        
        return pd.DataFrame(tools_info)
    
    def get_tool_usage_stats(self) -> pd.DataFrame:
        """Thống kê sử dụng công cụ"""
        stats = []
        
        for category, tool_names in self.registry.list_all_tools().items():
            for tool_name in tool_names:
                tool = self.registry.get_tool(tool_name)
                usage_count = len(tool.execution_log)
                
                if usage_count > 0:
                    total_time = sum(log['execution_time'] for log in tool.execution_log)
                    avg_time = total_time / usage_count
                else:
                    total_time = 0
                    avg_time = 0
                
                stats.append({
                    'Tool Name': tool.name,
                    'Category': tool.category,
                    'Usage Count': usage_count,
                    'Total Execution Time (s)': round(total_time, 3),
                    'Average Execution Time (s)': round(avg_time, 3)
                })
        
        return pd.DataFrame(stats).sort_values('Usage Count', ascending=False)
    
    def create_ml_pipeline(self, steps: List[Dict]) -> 'MLPipeline':
        """Tạo pipeline ML từ danh sách các bước"""
        return MLPipeline(self.registry, steps)
    
    def recommend_tools(self, problem_type: str, data_info: Dict) -> List[str]:
        """Đề xuất công cụ dựa trên loại bài toán và thông tin dữ liệu"""
        recommendations = []
        
        # Luôn đề xuất xử lý missing values nếu có
        if data_info.get('has_missing_values', False):
            recommendations.append('missing_value_handler')
        
        # Đề xuất outlier detection cho dữ liệu số
        if data_info.get('has_numeric_columns', False):
            recommendations.append('outlier_detector')
        
        # Đề xuất categorical encoding cho dữ liệu phân loại
        if data_info.get('has_categorical_columns', False):
            recommendations.append('categorical_encoder')
        
        # Đề xuất feature creation
        if data_info.get('feature_count', 0) < 20:  # Ít đặc trưng
            recommendations.append('feature_creator')
        
        # Đề xuất model trainer
        recommendations.append('model_trainer')
        recommendations.append('model_evaluator')
        
        return recommendations

class MLPipeline:
    """
    Class để tạo và thực thi pipeline ML tự động
    """
    
    def __init__(self, registry: MLToolRegistry, steps: List[Dict]):
        self.registry = registry
        self.steps = steps
        self.execution_log = []
        self.intermediate_results = {}
    
    def execute(self, initial_data: Dict) -> Dict:
        """Thực thi toàn bộ pipeline"""
        current_data = initial_data.copy()
        
        for i, step in enumerate(self.steps):
            step_name = step.get('name', f'step_{i}')
            tool_name = step['tool']
            params = step.get('params', {})
            
            print(f"Executing step {i+1}: {step_name} using {tool_name}")
            
            # Lấy công cụ từ registry
            tool = self.registry.get_tool(tool_name)
            if tool is None:
                raise ValueError(f"Tool {tool_name} not found in registry")
            
            # Thực thi công cụ
            try:
                start_time = time.time()
                
                # Truyền current_data vào params
                execution_params = {**params}
                for key, value in params.items():
                    if isinstance(value, str) and value.startswith('$'):
                        # Tham chiếu đến dữ liệu từ bước trước
                        data_key = value[1:]  # Bỏ ký tự '$'
                        execution_params[key] = current_data.get(data_key)
                
                result = tool.execute(**execution_params)
                execution_time = time.time() - start_time
                
                # Lưu kết quả
                current_data[f'{step_name}_result'] = result
                self.intermediate_results[step_name] = result
                
                # Log thực thi
                self.execution_log.append({
                    'step': step_name,
                    'tool': tool_name,
                    'execution_time': execution_time,
                    'success': True,
                    'error': None
                })
                
                print(f"  ✓ Completed in {execution_time:.2f}s")
                
            except Exception as e:
                error_msg = str(e)
                print(f"  ✗ Error: {error_msg}")
                
                self.execution_log.append({
                    'step': step_name,
                    'tool': tool_name,
                    'execution_time': 0,
                    'success': False,
                    'error': error_msg
                })
                
                # Quyết định có tiếp tục hay không
                if step.get('critical', True):
                    raise Exception(f"Critical step {step_name} failed: {error_msg}")
        
        return {
            'final_data': current_data,
            'intermediate_results': self.intermediate_results,
            'execution_log': self.execution_log
        }
    
    def get_execution_summary(self) -> pd.DataFrame:
        """Tóm tắt quá trình thực thi pipeline"""
        return pd.DataFrame(self.execution_log)

# Khởi tạo manager
ml_manager = MLToolsManager(ml_registry)

## Phần 6: Ví Dụ Minh Họa Sử Dụng Thư Viện

Hãy tạo một ví dụ hoàn chỉnh về cách sử dụng thư viện công cụ ML trong một quy trình khoa học dữ liệu:

In [ ]:
# Tạo dữ liệu mẫu để demo
def create_sample_dataset():
    """Tạo dữ liệu mẫu cho demo"""
    np.random.seed(42)
    n_samples = 1000
    
    # Tạo dữ liệu
    data = {
        'age': np.random.normal(35, 10, n_samples),
        'salary': np.random.normal(50000, 15000, n_samples),
        'experience': np.random.exponential(5, n_samples),
        'education': np.random.choice(['High School', 'Bachelor', 'Master', 'PhD'], n_samples, p=[0.3, 0.4, 0.2, 0.1]),
        'department': np.random.choice(['IT', 'HR', 'Finance', 'Marketing'], n_samples, p=[0.3, 0.2, 0.25, 0.25]),
        'performance_score': np.random.normal(75, 15, n_samples)
    }
    
    df = pd.DataFrame(data)
    
    # Thêm một số giá trị thiếu
    missing_indices = np.random.choice(df.index, size=int(0.1 * n_samples), replace=False)
    df.loc[missing_indices[:50], 'salary'] = np.nan
    df.loc[missing_indices[50:], 'experience'] = np.nan
    
    # Thêm một số outliers
    outlier_indices = np.random.choice(df.index, size=20, replace=False)
    df.loc[outlier_indices, 'salary'] = df.loc[outlier_indices, 'salary'] * 3
    
    # Tạo target variable (high performer: performance_score > 80)
    df['high_performer'] = (df['performance_score'] > 80).astype(int)
    
    return df

# Tạo dữ liệu mẫu
sample_data = create_sample_dataset()
print("Sample dataset created:")
print(f"Shape: {sample_data.shape}")
print(f"\nColumns: {list(sample_data.columns)}")
print(f"\nMissing values:")
print(sample_data.isnull().sum())
print(f"\nFirst 5 rows:")
print(sample_data.head())

In [ ]:
# Demo sử dụng các công cụ riêng lẻ
print("=== DEMO SỬ DỤNG CÁC CÔNG CỤ RIÊNG LẺ ===")

# 1. Xử lý missing values
print("\n1. Xử lý Missing Values:")
missing_handler = ml_registry.get_tool('missing_value_handler')
strategy = {
    'salary': 'median',
    'experience': 'mean'
}
data_cleaned = missing_handler.execute(sample_data, strategy)
print(f"Missing values after cleaning: {data_cleaned.isnull().sum().sum()}")

# 2. Phát hiện và xử lý outliers  
print("\n2. Xử lý Outliers:")
outlier_detector = ml_registry.get_tool('outlier_detector')
data_no_outliers, outlier_info = outlier_detector.execute(
    data_cleaned, 
    columns=['salary', 'age', 'experience'], 
    method='iqr', 
    action='cap'
)
print("Outlier information:")
for col, info in outlier_info.items():
    print(f"  {col}: {info['outlier_count']} outliers ({info['outlier_percentage']:.1f}%)")

# 3. Mã hóa categorical variables
print("\n3. Mã hóa Categorical Variables:")
categorical_encoder = ml_registry.get_tool('categorical_encoder')
data_encoded = categorical_encoder.execute(
    data_no_outliers,
    columns=['education', 'department'],
    method='onehot'
)
print(f"Columns after encoding: {data_encoded.shape[1]} (was {data_no_outliers.shape[1]})")

# 4. Tạo features mới
print("\n4. Tạo Features Mới:")
feature_creator = ml_registry.get_tool('feature_creator')
data_with_features = feature_creator.execute(
    data_encoded,
    operation='mathematical',
    columns=['age', 'salary', 'experience'],
    math_operations=['log', 'square']
)
print(f"Columns after feature creation: {data_with_features.shape[1]} (was {data_encoded.shape[1]})")

# 5. Huấn luyện mô hình
print("\n5. Huấn luyện Mô hình:")
model_trainer = ml_registry.get_tool('model_trainer')

# Chuẩn bị dữ liệu
X = data_with_features.drop(['high_performer', 'performance_score'], axis=1)
y = data_with_features['high_performer']

# Loại bỏ cột có giá trị NaN hoặc inf
X = X.select_dtypes(include=[np.number])
X = X.fillna(0)
X = X.replace([np.inf, -np.inf], 0)

print(f"Final feature matrix shape: {X.shape}")
print(f"Target distribution: {y.value_counts().to_dict()}")

# Huấn luyện mô hình
training_results = model_trainer.execute(
    X, y,
    problem_type='classification',
    models_to_try=['random_forest_clf', 'logistic_regression', 'xgboost_clf'],
    cv_folds=3
)

print("\nModel comparison:")
print(training_results['comparison_df'])
print(f"\nBest model: {training_results['best_model_name']}")
print(f"Best CV score: {training_results['best_model_info']['cv_mean']:.4f}")

# 6. Đánh giá mô hình
print("\n6. Đánh giá Mô hình:")
model_evaluator = ml_registry.get_tool('model_evaluator')
best_model = training_results['best_model']
X_test = training_results['X_test']
y_test = training_results['y_test']

y_pred = best_model.predict(X_test)
y_pred_proba = best_model.predict_proba(X_test)

evaluation_results = model_evaluator.execute(
    y_test, y_pred,
    problem_type='classification',
    y_pred_proba=y_pred_proba
)

print("Evaluation metrics:")
for metric, value in evaluation_results.items():
    if isinstance(value, float):
        print(f"  {metric}: {value:.4f}")
    elif metric not in ['classification_report', 'confusion_matrix']:
        print(f"  {metric}: {value}")

print("\n=== DEMO HOÀN THÀNH ===")

In [ ]:
# Demo sử dụng MLToolsManager
print("=== DEMO SỬ DỤNG ML TOOLS MANAGER ===")

# Liệt kê tất cả công cụ có sẵn
print("\n1. Danh sách tất cả công cụ:")
available_tools = ml_manager.list_available_tools()
print(available_tools)

# Tìm kiếm công cụ
print("\n2. Tìm kiếm công cụ với từ khóa 'feature':")
search_results = ml_manager.search_tools('feature')
print(search_results)

# Thống kê sử dụng
print("\n3. Thống kê sử dụng công cụ:")
usage_stats = ml_manager.get_tool_usage_stats()
print(usage_stats)

# Đề xuất công cụ
print("\n4. Đề xuất công cụ cho bài toán classification:")
data_info = {
    'has_missing_values': True,
    'has_numeric_columns': True,
    'has_categorical_columns': True,
    'feature_count': 5
}
recommendations = ml_manager.recommend_tools('classification', data_info)
print(f"Recommended tools: {recommendations}")

## Phần 7: Mở Rộng Thư Viện Với Công Cụ Mới

Một trong những ưu điểm quan trọng của thiết kế này là khả năng mở rộng. Hãy xem cách thêm một công cụ mới vào thư viện:

In [ ]:
# Ví dụ tạo công cụ mới: Feature Selector
class FeatureSelector(MLTool):
    """
    Công cụ lựa chọn đặc trưng sử dụng nhiều phương pháp khác nhau
    """
    
    def __init__(self):
        super().__init__(
            name="feature_selector",
            category=ToolCategory.FEATURE_ENGINEERING,
            description="Select best features using statistical tests, mutual information, or model-based methods"
        )
        self.supported_methods = {
            'correlation': self._correlation_selection,
            'mutual_info': self._mutual_info_selection,
            'chi2': self._chi2_selection,
            'lasso': self._lasso_selection,
            'random_forest': self._rf_selection
        }
        self.selected_features = {}
    
    def validate_input(self, X: pd.DataFrame, y: pd.Series, method: str) -> bool:
        """Kiểm tra tính hợp lệ của đầu vào"""
        if not isinstance(X, pd.DataFrame) or not isinstance(y, pd.Series):
            return False
        if len(X) != len(y):
            return False
        if method not in self.supported_methods:
            return False
        return True
    
    def execute(self, X: pd.DataFrame, y: pd.Series, 
                method: str = 'correlation',
                k_features: int = 10,
                threshold: float = 0.1) -> Tuple[pd.DataFrame, Dict]:
        """
        Lựa chọn đặc trưng tốt nhất
        
        Parameters:
        - X: DataFrame features
        - y: Series target
        - method: Phương pháp lựa chọn
        - k_features: Số lượng features muốn chọn
        - threshold: Ngưỡng cho một số phương pháp
        
        Returns:
        - Tuple của (DataFrame với features đã chọn, thông tin selection)
        """
        start_time = time.time()
        
        if not self.validate_input(X, y, method):
            raise ValueError("Invalid input parameters")
        
        # Chỉ xử lý cột số
        X_numeric = X.select_dtypes(include=[np.number])
        X_numeric = X_numeric.fillna(0)
        X_numeric = X_numeric.replace([np.inf, -np.inf], 0)
        
        # Áp dụng phương pháp lựa chọn
        selected_features, feature_scores = self.supported_methods[method](
            X_numeric, y, k_features, threshold
        )
        
        # Tạo DataFrame kết quả
        X_selected = X_numeric[selected_features]
        
        # Thông tin về quá trình lựa chọn
        selection_info = {
            'method': method,
            'original_features': X_numeric.shape[1],
            'selected_features': len(selected_features),
            'selected_feature_names': selected_features,
            'feature_scores': feature_scores,
            'reduction_percentage': (1 - len(selected_features) / X_numeric.shape[1]) * 100
        }
        
        execution_time = time.time() - start_time
        
        # Lưu thông tin
        self.selected_features[method] = selected_features
        
        # Log thực thi
        self.log_execution(
            inputs={'X_shape': X.shape, 'y_shape': y.shape, 'method': method, 'k_features': k_features},
            outputs={'selected_features': len(selected_features), 'X_selected_shape': X_selected.shape},
            execution_time=execution_time
        )
        
        return X_selected, selection_info
    
    def _correlation_selection(self, X: pd.DataFrame, y: pd.Series, k: int, threshold: float) -> Tuple[List[str], Dict]:
        """Lựa chọn dựa trên correlation với target"""
        correlations = X.corrwith(y).abs()
        correlations = correlations.sort_values(ascending=False)
        selected = correlations.head(k).index.tolist()
        scores = correlations.head(k).to_dict()
        return selected, scores
    
    def _mutual_info_selection(self, X: pd.DataFrame, y: pd.Series, k: int, threshold: float) -> Tuple[List[str], Dict]:
        """Lựa chọn dựa trên mutual information"""
        from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
        
        # Xác định loại bài toán
        if len(y.unique()) <= 10:  # Classification
            mi_scores = mutual_info_classif(X, y)
        else:  # Regression
            mi_scores = mutual_info_regression(X, y)
        
        # Tạo DataFrame để sort
        mi_df = pd.DataFrame({'feature': X.columns, 'score': mi_scores})
        mi_df = mi_df.sort_values('score', ascending=False)
        
        selected = mi_df.head(k)['feature'].tolist()
        scores = dict(zip(mi_df.head(k)['feature'], mi_df.head(k)['score']))
        
        return selected, scores
    
    def _chi2_selection(self, X: pd.DataFrame, y: pd.Series, k: int, threshold: float) -> Tuple[List[str], Dict]:
        """Lựa chọn dựa trên Chi-square test (chỉ cho classification)"""
        from sklearn.feature_selection import chi2
        
        # Đảm bảo tất cả giá trị không âm
        X_positive = X - X.min() + 1e-8
        
        chi2_scores, p_values = chi2(X_positive, y)
        
        # Tạo DataFrame để sort
        chi2_df = pd.DataFrame({
            'feature': X.columns, 
            'chi2_score': chi2_scores,
            'p_value': p_values
        })
        chi2_df = chi2_df.sort_values('chi2_score', ascending=False)
        
        selected = chi2_df.head(k)['feature'].tolist()
        scores = dict(zip(chi2_df.head(k)['feature'], chi2_df.head(k)['chi2_score']))
        
        return selected, scores
    
    def _lasso_selection(self, X: pd.DataFrame, y: pd.Series, k: int, threshold: float) -> Tuple[List[str], Dict]:
        """Lựa chọn dựa trên Lasso regularization"""
        from sklearn.linear_model import Lasso
        from sklearn.preprocessing import StandardScaler
        
        # Chuẩn hóa dữ liệu
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        # Fit Lasso
        lasso = Lasso(alpha=threshold, random_state=42)
        lasso.fit(X_scaled, y)
        
        # Lấy coefficients
        coef_abs = np.abs(lasso.coef_)
        
        # Tạo DataFrame để sort
        coef_df = pd.DataFrame({
            'feature': X.columns,
            'coefficient': coef_abs
        })
        coef_df = coef_df.sort_values('coefficient', ascending=False)
        
        # Chọn features có coefficient > 0
        non_zero_features = coef_df[coef_df['coefficient'] > 0]
        selected = non_zero_features.head(k)['feature'].tolist()
        scores = dict(zip(non_zero_features.head(k)['feature'], non_zero_features.head(k)['coefficient']))
        
        return selected, scores
    
    def _rf_selection(self, X: pd.DataFrame, y: pd.Series, k: int, threshold: float) -> Tuple[List[str], Dict]:
        """Lựa chọn dựa trên feature importance từ Random Forest"""
        from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
        
        # Xác định loại bài toán
        if len(y.unique()) <= 10:  # Classification
            rf = RandomForestClassifier(n_estimators=100, random_state=42)
        else:  # Regression
            rf = RandomForestRegressor(n_estimators=100, random_state=42)
        
        rf.fit(X, y)
        
        # Lấy feature importance
        importance_df = pd.DataFrame({
            'feature': X.columns,
            'importance': rf.feature_importances_
        })
        importance_df = importance_df.sort_values('importance', ascending=False)
        
        selected = importance_df.head(k)['feature'].tolist()
        scores = dict(zip(importance_df.head(k)['feature'], importance_df.head(k)['importance']))
        
        return selected, scores

# Đăng ký công cụ mới
ml_registry.register_tool(FeatureSelector())

print("Đã thêm công cụ Feature Selector vào thư viện!")
print("\nDanh sách công cụ mới:")
print(ml_manager.list_available_tools())

## Kết Luận

Thư viện công cụ học máy (Machine Learning Tools Library) trong AutoKaggle là một thành phần quan trọng và được thiết kế rất tốt. Thông qua notebook này, chúng ta đã học được:

### Những Điểm Chính:

1. **Tổ Chức Có Hệ Thống**: Thư viện được tổ chức thành 3 danh mục chính (Data Cleaning, Feature Engineering, Model Building) với các công cụ chuyên biệt cho từng giai đoạn

2. **Thiết Kế Nhất Quán**: Tất cả công cụ đều kế thừa từ MLTool base class, đảm bảo interface nhất quán và dễ sử dụng

3. **Khả Năng Mở Rộng**: Kiến trúc Registry pattern cho phép dễ dàng thêm công cụ mới mà không cần sửa đổi code hiện có

4. **Tự Động Hóa Cao**: Các công cụ được thiết kế để tự động hóa các tác vụ phức tạp, từ xử lý dữ liệu đến huấn luyện mô hình

5. **Logging và Monitoring**: Mỗi công cụ đều có khả năng ghi log quá trình thực thi, giúp debug và tối ưu hóa

### Lợi Ích Của Thiết Kế:

- **Tái Sử Dụng**: Các công cụ có thể được sử dụng độc lập hoặc kết hợp trong pipeline
- **Tin Cậy**: Các công cụ đã được kiểm thử và validate, đảm bảo tính đúng đắn
- **Hiệu Quả**: Tự động hóa các tác vụ lặp đi lặp lại, tiết kiệm thời gian
- **Dễ Bảo Trì**: Kiến trúc module hóa giúp dễ dàng cập nhật và sửa lỗi

### Ứng Dụng Thực Tế:

Thiết kế này có thể được áp dụng cho:
- Xây dựng platform AutoML
- Tạo công cụ hỗ trợ data scientist
- Phát triển hệ thống ML-as-a-Service
- Giáo dục và đào tạo về khoa học dữ liệu

Thư viện công cụ ML trong AutoKaggle thể hiện một cách tiếp cận chuyên nghiệp và có hệ thống để tự động hóa quy trình khoa học dữ liệu, đồng thời duy trì tính linh hoạt và khả năng mở rộng.